In [1]:
import warnings
warnings.filterwarnings('ignore') # Import warnings to avoid SSL cert error

import ast, sys, string, re, os, json # from python standard library
from ast import literal_eval
from collections import OrderedDict

import pandas as pd # data formatting and cleaning
import numpy as np
from mlxtend.frequent_patterns import apriori # specific algo. we're using to compute association rules
from mlxtend.frequent_patterns import association_rules 

# PART 1: DATA

### Import data and clean

In [2]:
sales_data = pd.read_excel('France Retail.xlsx')
sales_data = sales_data.drop(columns=['InvoiceDate', 'UnitPrice', 'CustomerID', 'Country'])
sales_data = sales_data[sales_data['Description'] != 'POSTAGE'] # remove postal payments from orders; redundant
sales_data.Description = sales_data.Description.str.strip() #remove leading & trailing characters
sales_data.dropna(axis = 0, subset=['InvoiceNo'], inplace = True)

# He converts invoice number to string so he can strip postal and credit transactions from basket
sales_data.InvoiceNo = sales_data['InvoiceNo'].astype('str')
sales_data = sales_data[~sales_data['InvoiceNo'].str.contains('C')] # removes credit card Tx
sales_data['InvoiceNo'] = pd.to_numeric(sales_data['InvoiceNo']).astype(np.int64)
# MAKE MY OWN STOCK IDs SINCE SALES DATA IS POOR QUALITY
sales_data['Stock_ID'] = sales_data.groupby(sales_data['Description']).grouper.group_info[0]
sales_data = sales_data.drop(columns=['StockCode'])

In [3]:
stock_lookup = sales_data.drop(columns=['InvoiceNo', 'Quantity'])
stock_lookup['Stock_ID'] = stock_lookup['Stock_ID'].drop_duplicates()
stock_lookup['Description'] = stock_lookup['Description'].drop_duplicates()
stock_lookup.dropna(axis = 0, subset=['Stock_ID'], inplace = True)
sales_data.dropna(axis = 0, subset=['Description'], inplace = True)
stock_lookup['Stock_ID'] = stock_lookup['Stock_ID'].astype(int)

In [4]:
stock_lookup.reset_index(inplace=True, drop=True)
stock_lookup

,Description,Stock_ID
0,ALARM CLOCK BAKELIKE PINK,64
1,ALARM CLOCK BAKELIKE RED,65
2,ALARM CLOCK BAKELIKE GREEN,61
3,PANDA AND BUNNIES STICKER SHEET,857
4,STARS GIFT TAPE,1337
...,...,...
1557,DOORMAT FAIRY CAKE,375
1558,SKULLS STORAGE BOX LARGE,1286
1559,SNACK TRAY I LOVE LONDON,1307
1560,SET OF 6 RIBBONS PERFECTLY PRETTY,1221


### Compute association rules

In [5]:
# COMPUTE RULES 
def generate_rules(transactions = sales_data): 
    """This nesting function takes your transaction data and returns association rules. 
    This nested functions includes like data formatting, rule computing, and rule table processing."""

    def encoder(x):
        """Necessary to process data in right format for our mlxtend package"""
        if x <= 0:
            return 0
        else: # if x >= 1
            return 1

    def create_transaction_data(transactions): 
        """Function takes your transaction data as an argument, and turns into transaction object that market basket analysis can be 
        conducted on. It requires, at minimum, a transaction ID and ID of item."""    
        transaction_basket = transactions.groupby(['InvoiceNo', 'Stock_ID'])['Quantity'].sum()
        transaction_basket = transaction_basket.unstack()
        transaction_basket = transaction_basket.reset_index()
        transaction_basket = transaction_basket.fillna(0)
        transaction_basket = transaction_basket.set_index('InvoiceNo')
        transaction_basket = transaction_basket.applymap(encoder)
        return(transaction_basket)

    def compute_rules(basket):
        """Computes association rules, which are necessary to make team suggestions. Returns rules as dataframe"""
        
        frequent_itemsets = apriori(basket, 
                       min_support= 0.001, # smaller support, more expensive and time consuming to run
                            # unfortunately, for our dataset, a larger support threshold means no rules are accepted 
                       use_colnames= True, 
                       max_len = 2)

        rules = association_rules(frequent_itemsets, 
                             metric = 'lift',
                             min_threshold=1) 
        
        rules = rules.sort_values(by='lift', ascending = False)
        rules = rules.reset_index()
        rules = rules[rules['antecedents'].apply(lambda x: len(x) < 2)] # remove ANTECEDENTS/LHS with more than 1 products
        # REASON WE DO THIS IS BECAUSE WE ONLY ALLOW ONE PRODUCT TEAM TO BE SELECTED AT A TIME!
        rules['LHS'] = ([tuple(x) for x in list(rules.antecedents)]) 
        # convert from frozen set to tuple: need to modify but frozensets are immutable
        rules.LHS = rules.LHS.astype(str) # convert to string so we can remove punctuation
        rules.LHS = rules.LHS.str.split(",", expand=True) # split string on comma so we can remove punctuation 
        rules.LHS= rules.LHS.str.replace(re.escape('('), '') # remove punctuation
        rules.LHS = (rules.LHS).astype(int) # convert to integer
        
        rules['RHS'] = ([tuple(x) for x in list(rules.consequents)])
        rules = rules.drop(columns={'index', 'antecedent support', 'consequent support', 'leverage', 'conviction', 'antecedents', 'consequents'})
        # strip the comma from LHS: it's only ever going to be one item so can remove the comma and leave it as integer variable
        
        
        rules['support'] = rules['support'].round(decimals=5) # need 5 dp otherwise they all get same support
        rules['lift'] = rules['lift'].astype(int) # can just have this as an integer, no need for precise decimal points
        rules['RHS'] = rules['RHS'].astype(str) # convert tuple field to string, since it's a varchar field in DB table

        # CREATING A COUNT COLUMN TO OBSERVE HOW MANY TIMES WE OBSERVE AN ASSOCIATION RULE IN DATA
        counts = rules['support']*len(basket) # support is simply no. of occurences divided by total transactions
        rules.insert(column = 'count', value = counts, loc = 5) # add count column to rules_count df
        rules['count'] = rules['count'].astype(int) 
        rules = rules.sort_values(['count'], ascending = [False])
        rules = rules[rules['count'] > 2] 
        # convert all col names to upper case
        rules.columns = map(str.upper, rules.columns)

        return(rules)
                                                                              
    basket = create_transaction_data(sales_data) # format transaction data in right format for mlxtend package 
    rules = compute_rules(basket) # compute association rules, which are used to make suggestions
    return(rules)
                                                                                  
                                                                              

In [6]:
rules = generate_rules(sales_data)

In [530]:
sales_data[sales_data['Stock_ID'] == 1266].head(1)

,InvoiceNo,Description,Quantity,Stock_ID
221,538008,SET/6 RED SPOTTY PAPER CUPS,12,1266


In [7]:
rules

,SUPPORT,CONFIDENCE,LIFT,LHS,RHS,COUNT
150235,0.12403,0.888889,6,1266,"(1267,)",47
150232,0.12403,0.960000,6,1267,"(1266,)",47
161669,0.10336,0.740741,5,1266,"(1247,)",40
157229,0.10336,0.800000,5,1267,"(1247,)",40
174877,0.10594,0.611940,4,956,"(953,)",40
...,...,...,...,...,...,...
206551,0.01034,0.235294,2,241,"(1026,)",4
96417,0.01034,0.400000,17,848,"(44,)",4
226051,0.01034,0.108108,1,1370,"(641,)",4
138920,0.01034,0.166667,8,972,"(306,)",4


In [8]:
rules.to_csv(r'C:\Users\NJM\Desktop\Computing\Deployment\Practice run\Recommendation\rules.csv', 
          index = False, 
          header = True)

# PART 2: SUGGESTIONS

In [9]:
# Read in rules
rules = pd.read_csv('rules.csv', converters={"RHS": ast.literal_eval}) 

In [10]:
def shopping_suggestions(new_item, rules = rules):    
    
    def query_ID(new_item):
        """Uses stock lookup table to check what stock ID corresponds to chosen product description"""
        ID = stock_lookup[stock_lookup['Description'] == new_item].iloc[0][1]
        return int(ID)     

    def match(new_item, rules):
        """Returns rules specific to new item chosen"""
        matched = rules[(rules.LHS.isin(new_item))]
        matched = matched[~matched.index.duplicated(keep='last')]  # remove duplicate rows based on index (the index used in sql table)
        matched = matched.sort_index()
        return(matched)

    def output_consequents(matched_rules):
        """Looks at rules specific to our new PWR, then outputs 'consequents'; these are our suggestions WITHOUT any of our specific hard-coded rules applied"""
        consequent_teams = (x for x in list(matched_rules.RHS)) 
        consequent_teams
        recommendations =[] 
        for i in consequent_teams: # for each tuple in the list
            for j in i: # for each element (i.e. product team) in the tuple
                recommendations.append(j) # add product team to unfiltered recommendations
                recommendations = list(OrderedDict.fromkeys(recommendations)) # keep unique teams
                recommendations = recommendations[0:3]
        return(recommendations) 

    def query_name_info(recommendations):
        recommendations = recommendations
        suggestion_string = None
        info_as_df = stock_lookup[stock_lookup['Stock_ID'].isin(recommendations)]

        try:
            suggestion_string = ('Customers also bought the following: ' + 
                                 info_as_df.iloc[0][0] + 
                                 ', ' 
                                 + info_as_df.iloc[1][0] + ', '
                                 + info_as_df.iloc[2][0] + '.')
        except KeyError: # in the event only ONE team is suggested, the error is ignored
            suggestion_string = ('Customers also bought ' +
                                        (info_as_df.iloc[0][0] + '. '))

        return(json.dumps(suggestion_string, separators=(". "))) # use . and a space to separate objects in JSON string

    new_item = new_item
    new_item = [query_ID(new_item)]
    rules = rules 
    matched_rules = match(new_item, rules)
    recommendations = output_consequents(matched_rules)
    try:   
        suggestion_string = query_name_info(recommendations)
    except: # FOR IF WE HAVE NO IDs IN recommendations This would occur if no initial teams are chosen!
        suggestion_string = 'No relevant team recommendations available'
    return(suggestion_string)


In [11]:
shopping_suggestions(new_item = "SET/6 RED SPOTTY PAPER CUPS", rules = rules)

'"Customers also bought the following: SET/6 RED SPOTTY PAPER PLATES, PACK OF 6 SKULL PAPER CUPS, SET/20 RED RETROSPOT PAPER NAPKINS."'

In [12]:
shopping_suggestions(new_item="CHILDS BREAKFAST SET DOLLY GIRL", rules=rules)

'"Customers also bought the following: CHILDS BREAKFAST SET SPACEBOY, PLASTERS IN TIN WOODLAND ANIMALS, PLASTERS IN TIN SPACEBOY."'

In [579]:
stock_lookup[stock_lookup['Description'] == "CHILDS BREAKFAST SET DOLLY GIRL"]

,Description,Stock_ID
62,CHILDS BREAKFAST SET DOLLY GIRL,266


In [578]:
stock_lookup.loc[62]

Description    CHILDS BREAKFAST SET DOLLY GIRL
Stock_ID                                   266
Name: 62, dtype: object

# FOR NETWORK EXPORT ONLY

In [22]:
sales_data = sales_data.rename(columns={'Description': "PRODUCT_NAME"})
sales_data = sales_data.rename(columns={'Stock_ID': "PROD_ID"})
sales_data.to_csv(r'C:\Users\NJM\Desktop\Computing\Deployment\Practice run\Recommendation\clean_transaction_data.csv', 
                  index = False, 
                  header = True, encoding='UTF-8-sig')

# PART 3: DEPLOYMENT

* imports including flask
* model code (this'd refer to shopping suggestion)
* Insurance example had fields (dependent variable). I guess for this one, we'd have drop down menu they select product from
* app route that takes home html 
* app route that makes predictions. Usually it's @app.route('/predict', methods = ['POST']) but not sure what it'll be for this one

<br><br>
* https://stackoverflow.com/questions/59781313/is-possible-to-reuse-import-code-in-python
* rules would already need to be stored, then simply referred to in git repo deployment
* https://stackoverflow.com/questions/52443855/taking-data-from-drop-down-menu-using-flask
* https://www.reddit.com/r/flask/comments/2fi58q/af_populate_dropdown_from_python_list/ 
* 